In [1]:
## General librairies
import pandas as pd
import numpy as np
import _pickle as pickle
import matplotlib.pyplot as plt
import random
import sys
import time
from copy import deepcopy
import os

from keras.utils import np_utils
from sklearn.model_selection import train_test_split


import dateutil.parser

Using TensorFlow backend.


In [2]:
specs = 'specs.csv'
test = 'test.csv'
train = 'train.csv'
train_labels = 'train_labels.csv'
subs = 'sample_submission_exemple.csv'

In [3]:
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

def relative_time(x):
    x1 = []
    for elt in x:
        x1.append((elt-x[0]).item()/1000000000)
    return x1

def categorise(x):
    dico = {}
    count = 0
    for elt in x:
        if not(elt in dico):
            dico[elt] = count
            count += 1
    return dico

def padding( dataset, n):
    d = list(np.zeros(len(dataset)))
    c = 0
    count = 0
    for elt in dataset:
        
        if count % 100 == 0:
            print(count)
        u = elt.shape[0]
        
        if u<200:
            c += 1
        
        if u > n:
            d[count] = elt[-n:]
        else:
            a = np.zeros(((n-u), elt.shape[1]))
            elt = np.concatenate([a, elt], axis = 0)
            d[count] = elt
        count += 1
    return d

def build_line(df, installation_id, game_session):
    df1 = df[(df['installation_id']==installation_id)&(df['date']<=df[(df['installation_id']==installation_id)&(df['game_session']==game_session)]['date'].iloc[0])]
    dico = categorise(df1['game_session'].unique())
    df1 = df1.replace({'game_session' : dico})
    x = relative_time(df1['date'].values)
    
    df1['time_delta'] = x
    
#     df1 = df1.drop_duplicates(subset = ['event_id'])
    
    cats_to_keep = ['game_session','time_delta','title','type', 'world',  'event_count', 'game_time', 'event_code']
    df1 = df1[cats_to_keep]
    return df1

def format_input(X):
    return [X[:,:,[0,1,5,6]], X[:,:,2].reshape((X.shape[0], X.shape[1],1)), X[:,:,3].reshape((X.shape[0], X.shape[1],1)), X[:,:,4].reshape((X.shape[0], X.shape[1],1)),X[:,:,7].reshape((X.shape[0], X.shape[1],1))]


#### Loading Data

In [4]:
df = pd.read_csv(train)
# labels = pd.read_csv(train_labels)
# dtitle, dtype, dworld, devent = load('dicos')

In [50]:
df[2300] = 1

In [6]:
df['event_data'].value_counts()

{"event_code": 2000, "event_count": 1}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            183676
{"version":"1.0","event_count":1,"game_time":0,"event_code":2000}                                                                                                                                                                                                                                                                                                                                                                              

In [7]:
import json
from tqdm import tqdm_notebook as tqdm
dico = {}
count = 0
for elt in tqdm(df['event_data'].values):
    a = json.loads(elt)
    for key in a:
        if not(key in dico):
            dico[key] = a[key]
    

In [8]:
dico

{'event_code': 2000,
 'event_count': 1,
 'version': '1.0',
 'game_time': 0,
 'description': "Let's build a sandcastle! First, fill up your mold with sand! You can use the shovel here. The mold gives the sand its shape!",
 'identifier': 'Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape',
 'media_type': 'audio',
 'total_duration': 6758,
 'duration': 6919,
 'coordinates': {'x': 583, 'y': 605, 'stage_width': 1015, 'stage_height': 762},
 'castles_placed': [],
 'molds': [0, 0, 0],
 'size': 0,
 'sand': 1,
 'filled': False,
 'level': 0,
 'round': 0,
 'movie_id': 'scrubadub_intro',
 'round_target': {'size': 3, 'type': 'Tub', 'animal': 'hog'},
 'options': [0, 3],
 'animals': ['hog', 'chicken'],
 'item_type': 'Tub',
 'position': 2,
 'animal': 'hog',
 'correct': True,
 'misses': 0,
 'holding_shell': False,
 'has_water': False,
 'shells': [2, 3, 1],
 'holes': [2, 3],
 'shell_size': 1,
 'hole_position': 1,
 'cloud': 'middle',
 'cloud_size': 3,
 'water_level': 0,
 'time_played': 1376,
 'houses': [4, 3, 

In [52]:
df.columns

Index([       'event_id',    'game_session',       'timestamp',
            'event_data', 'installation_id',     'event_count',
            'event_code',       'game_time',           'title',
                  'type',           'world',              2300],
      dtype='object')

In [48]:
df.groupby(['installation_id'])['Clip'].transform('count')

KeyError: 'Column not found: Clip'

In [8]:
df['type'].value_counts()['Game']

5795293

In [5]:
df[(df['type']=='Game')&(df['event_code']==4100)]['title'].unique()

array(['Air Show', 'Pan Balance'], dtype=object)

In [37]:
df[df['type']=='Game']['title'].unique()

array(['Scrub-A-Dub', 'Dino Drink', 'All Star Sorting', 'Air Show',
       'Crystals Rule', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Pan Balance', 'Happy Camel', 'Leaf Leader'], dtype=object)

In [5]:
df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [28]:
df[df['type']=='Clip']

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
1103,27253bdc,091587ed87b9637c,2019-09-06T18:10:51.183Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
1355,27253bdc,3fce4f09769ff0b7,2019-09-06T18:18:11.250Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Tree Top City - Level 1,Clip,TREETOPCITY
1356,27253bdc,04168b61c0650977,2019-09-06T18:18:42.392Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Ordering Spheres,Clip,TREETOPCITY
...,...,...,...,...,...,...,...,...,...,...,...
11340971,27253bdc,e131c218f77fbeab,2019-10-10T15:08:23.374Z,"{""event_code"": 2000, ""event_count"": 1}",fffc0583,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
11341038,27253bdc,6ad8ab25003ef1b0,2019-10-10T15:12:17.364Z,"{""event_code"": 2000, ""event_count"": 1}",fffc0583,1,2000,0,Crystal Caves - Level 2,Clip,CRYSTALCAVES
11341039,27253bdc,71904dca23b982c6,2019-10-10T15:13:04.713Z,"{""event_code"": 2000, ""event_count"": 1}",fffc0583,1,2000,0,Crystal Caves - Level 3,Clip,CRYSTALCAVES
11341040,27253bdc,2267b18f069e875a,2019-10-10T15:13:54.545Z,"{""event_code"": 2000, ""event_count"": 1}",fffc0583,1,2000,0,Crystal Caves - Level 3,Clip,CRYSTALCAVES


In [33]:
df[['installation_id', 'world']].groupby(['installation_id', 'world']).sum().to_csv('worlds.csv')

In [8]:
df1 = df[df['event_code']==3010][['title', 'event_data', 'type', 'world']]#.groupby(['title', 'event_data']).sum()

In [10]:
import json
elt = df1.iloc[0]['event_data']
json.loads(elt)['description']

def load_json(x):
    return json.loads(x)['description']

df1['description'] = df1['event_data'].apply(load_json)

In [12]:
df1 = df1[['title', 'description', 'type', 'world']]

In [14]:
df1 = df1.groupby(['title', 'description', 'type', 'world']).sum()

In [20]:
df1.to_csv('activities.csv', sep = ';')

In [ ]:
df[(df['title'] == 'Sandcastle Builder (Activity)')&(df['event_code']==3010)]

In [ ]:
df['world'].value_counts()

In [ ]:
df

In [ ]:
df[df['event_id'] == '17113b36']['event_code'] = 4100

In [ ]:
df[(df['type'] == 'Assessment')&(df['event_code']==4100)].groupby(['game_session']).sum()

In [ ]:
df[['game_session', 'installation_id']].groupby(['game_session', 'installation_id']).sum()

In [ ]:
df[df['type']=='Clip']

In [ ]:
df[(df['installation_id']=='00279ac5')&(df['game_session']=='346d5f57cf40bac0')]#.iloc[-6]['event_data']

In [ ]:
df[(df['installation_id']=='00279ac5')&(df['game_session']=='346d5f57cf40bac0')]['event_code'].value_counts()

In [ ]:
df[df['event_id'] == '2b9272f4']

In [ ]:
df[df['type']=='Activity']['event_code'].value_counts().sort_index()


In [ ]:
list(df['title'].unique())

In [ ]:
labels = pd.read_csv(train_labels)

In [ ]:
df.head()

#### Adding some formating to dataframe

In [ ]:
ids = df[df['type']=='Assessment']['installation_id'].unique()
df = df[df.installation_id.isin(ids)]
df = df[df.installation_id.isin(labels.installation_id.unique())]

dtitle, dtype, dworld, devent = load('dicos')

dtitle = categorise(df['title'])
dtype = categorise(df['type'])
dworld = categorise(df['world'])
devent = categorise(df['event_code'])

save((dtitle, dtype, dworld, devent), 'dico')

df = df.replace({'title' : dtitle})
df = df.replace({'type' : dtype})
df = df.replace({'world' : dworld})
df = df.replace({'event_code' : devent})

# df['date'] = df['timestamp'].apply(dateutil.parser.parse)
# df = df.sort_values(by = ['date'], ascending =True)
# df = df.reset_index()

# df = df.reset_index()

In [ ]:
from multiprocess import Pool
def f(x):
    import dateutil.parser
    return dateutil.parser.parse(x)

if __name__ == '__main__':
    p = Pool(8)
    vect = p.map(f, df['timestamp'].values)

df['date'] = vect
df = df.sort_values(by = ['date'], ascending =True)
df = df.reset_index()

df = df.reset_index()

In [ ]:
save(df, 'time train')

In [ ]:
df = load('time train')

In [ ]:
df['event_count'] = (df['event_count'].values - df['event_count'].values.mean())/df['event_count'].values.std()
df['game_time'] = (df['game_time'].values - df['game_time'].values.mean())/df['game_time'].values.std()

In [ ]:
df = df.sort_values(by = ['date'], ascending =True)
df = df.sort_values(by = ['installation_id'], ascending =True)

dico_id = {}

vect = df['installation_id'].values
count = 0
for elt in vect:    
    if count == 0:
        val1 = 0
        mem = elt
    
    if elt != mem:
        val2 = count
        dico_id[mem] = [val1, val2]
        val1 = count
    
    if count == vect.shape[0]-1:
        dico_id[mem] = [val1, vect.shape[0]]
    
    mem = elt
    count += 1

In [ ]:
per_id = list(np.zeros(3614))

count = 0
for elt in labels['installation_id'].unique():
    per_id[count] = labels[labels['installation_id']==elt]
    count += 1



dataset = []

for i in range(len(per_id)):
    print(i)
    temp = list(np.zeros(per_id[i].shape[0]))
    rows = dico_id[per_id[i].iloc[0]['installation_id']]
    df1 = df.iloc[rows[0]:rows[1]]
    df1 = df1.sort_values(by = ['date'], ascending =True)
#     df1 = df1.drop_duplicates(subset = ['event_id'])
#     print(df1)
#     df1 = df[df['installation_id'] == per_id[i].iloc[0]['installation_id']]
#     print(df1.head())
    for j in range(len(temp)):
        installation_id = per_id[i].iloc[j]['installation_id']
        game_session = per_id[i].iloc[j]['game_session']
        df2 = build_line(df1, installation_id, game_session)

        temp[j] = df2.values
    dataset.extend(temp)

In [ ]:
save(dataset, 'non_padded')

In [ ]:
test = load('test')
# ids = df['installation_id'].unique()

In [ ]:
test[['title', 'type', 'world', 'event_count']].groupby(['title', 'type', 'world']).sum()

In [ ]:
test['installation_id'].unique()

In [ ]:
id1 = '167fe32c'
test[test['installation_id'] == id1]

In [ ]:
test[(test['installation_id'] == id1)&(test['type']=='Assessment')]

In [ ]:
b = test[test['type'] == 'Assessment'][['installation_id','game_time','event_count']].groupby(['installation_id']).max()

In [ ]:
b['game_time'].values.max()

In [ ]:
test['type'].unique()

In [ ]:
test[test['event_code'] == 4230]['event_id'].unique()

In [ ]:
df[df['event_code']==4100]#.iloc[1]['event_data']#[11:15]


In [ ]:
import json
# train_df['event_data'].apply(lambda x: json.loads(x).get('event_count'))

json.loads(df[df['event_code']==4100].iloc[1]['event_data'])['correct']

In [ ]:
import ast

ast.literal_eval(df[df['event_code']==4100].iloc[0]['event_data'])

In [ ]:
dico = {}

In [ ]:
type(dico)

In [ ]:
df[df['type']=='Activity']['event_code'].value_counts()

In [ ]:
test[test['type']=='Clip']['title'].unique()

In [ ]:
test[test['type']=='Game']['title'].unique()

In [ ]:
test[test['type']=='Activity']['title'].unique()

In [ ]:
test[test['type']=='Assessment']['title'].unique()

In [ ]:
test['type'].value_counts()